Vamos a hacer un _mixed_ recommendation system, en el que simplemente se hacen $k$ recomendaciones de cada uno, y se combinan las dos listas para obtener $k$ recomendaciones.

- Si un item está en las dos listas, se escoge sí o sí (sin importar su posición en la lista)
- Los huecos restantes se rellenan con un item de cada lista alternativamente

Es decir, si R1 recomienda {A,B,C,D,E} y R2 recomienda {A,D,F,G,H,I}, el recomendador mixto recomendaría {A,D,B,F,C}. A y D se recomiendan porque están en ambas, y luego se recomendaría el restante de cada uno.

In [2]:
class HybridRecommendation:
    def __init__(self, train, test, *, lightgcn_hparams=None, nlp_hparams=None):
        self.train = train
        self.test = test
        self.lightgcn = self._prepare_lightgcn(lightgcn_hparams)
        self.nlp = self._prepare_nlp(nlp_hparams)

    def _prepare_lightgcn(self, hparams):
        dataloader = ImplicitCF(train=self.train, test=self.test, seed=SEED)
        return LightGCNCustom(data=dataloader, hparams=hparams)

    def _prepare_nlp(self, hparams):
        return ...

    def fit():
        self.lightgcn.fit()
        self.nlp.fit()
    
    def recommend_k_items(
        self, test, top_k=10, sort_top_k=True, remove_seen=True, use_id=False, recommend_from=None,
    ):
        ...